<a href="https://colab.research.google.com/github/sebasbrowar/mcd/blob/main/api_mal_documentada_SBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Usando la API para obtener datos sobre personas desaparecidas del RNPDNO


<p> Julio Waissman Vilanova </p>



<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/integracion/python/RNPDNO-API.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

In [ ]:
import os
import sys
import requests
import datetime

import pandas as pd
import json


## Calentando motores

Para descargar los datos, vamos a consultar directamente la base de datos pública del [Registro Nacional de Personas Desaparecidas y No Localizadas (RNPDNO)](https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index).

El RNPDNO no tiene una API tal cual, sin embargo, [Pablo Reyes Moctezuma](https://github.com/pablorm296) encontró una manera de extraer la información usando la librería `request` de python. La API que, me imagino, el extrajo a punta de prueba y error la documento en [este archivo en markdown](https://github.com/pablorm296/ScrapperRNPDNO/blob/master/Test/API.md). Un chambón.

Vamos air la usando poco a poco, empecemos por tratar de encontrar en el catálogo los indices de estados, municipios y colonias.

In [ ]:
API_HOST = "https://versionpublicarnpdno.segob.gob.mx/"
API_SOCIODEOGRAFICOS_ROOT = "Sociodemografico/"
API_CATALAGO_ROOT = "Catalogo/"

ENDPOINT_CATALOGO_EDO = "Estados/"
ENDPOINT_CATALOGO_MUN = "Municipios/"
ENDPOINT_CATALOGO_COL = "Colonias/"

# Before doing anything, we must make a dummy request to the index in order to get the propper cookies
main_session = requests.Session()
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index")
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/ContextoGeneral")

<Response [200]>

Los identificadores de los estados:

In [ ]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_EDO

r = main_session.post(TARGET_URL)
estados_id = pd.json_normalize(r.json(),)
estados_id.columns = ['Valor', 'Estado']
estados_id

,Valor,Estado
0,0,--TODOS--
1,1,AGUASCALIENTES
2,2,BAJA CALIFORNIA
3,3,BAJA CALIFORNIA SUR
4,4,CAMPECHE
5,7,CHIAPAS
6,8,CHIHUAHUA
7,9,CIUDAD DE MEXICO
8,5,COAHUILA
9,6,COLIMA


y ahora los municipios de Sonora

In [ ]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_MUN
DATA = {"idEstado": "26"}

r = main_session.post(TARGET_URL, data = DATA)
mun_son_id = pd.json_normalize(r.json())
mun_son_id.columns = ['Valor', 'Municipio']
mun_son_id

,Valor,Municipio
0,0,--TODOS--
1,1,ACONCHI
2,2,AGUA PRIETA
3,3,ALAMOS
4,4,ALTAR
...,...,...
69,65,TUBUTAMA
70,66,URES
71,67,VILLA HIDALGO
72,68,VILLA PESQUEIRA


y por último los identificadores de las colonias del municipio de Hermosillo

In [ ]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_COL
DATA = {"idEstado": "26", "idMunicipio": "30"}

r = main_session.post(TARGET_URL, data = DATA)
col_hmo_id = pd.json_normalize(r.json())
col_hmo_id.columns = ['Valor', 'Colonia']
col_hmo_id

,Valor,Colonia
0,0,--TODAS--
1,347025,22 DE SEPTIEMBRE
2,347004,26 DE OCTUBRE
3,347026,4 DE MARZO
4,347027,4 OLIVOS
...,...,...
631,347358,VISTA DEL LAGO
632,347359,Y
633,347582,ZACATON
634,347384,ZAMORA


## Sociodemográficos totales

`Con este `endpoint` se pueden consultar resúmenes generales de la información que se pide. recuerda de revisar los catálogos.

Hay dos variables cuyos valores posibles son los siguientes:

**idEstatusVictima**:
- "0" PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS
- "2" PERSONAS LOCALIZADAS CON VIDA
- "3" PERSONAS LOCALIZADAS SIN VIDA- "4" PERSONAS DESAPARECIDAS
- "5" PERSONAS NO LOCALIZADAS
- "6" PERSONAS LOCALIZADAS
- "7" PERSONAS DESAPARECIDAS Y NO LOCALIZADAS

**idHipotesisNoLocalizacion**:
- "0" --TODAS--
- "1" ACCIDENTE
- "2" CATÁSTROFE
- "3" NO LOCALIZACIÓN VOLUNTARIA
- "4" NO LOCALIZACIÓN INVOLUNTARIA
- "5" SE DESCONOCE

Veamos como funciona pidiendo información de Sonora y de Hermosillo. Empecemos por Sonora

In [ ]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "Totales"

DATA = {
  "titulo":"",
  "subtitulo": "",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
  "idDelito":"0"
}
r = main_session.post(TARGET_URL, json = DATA)

resumen_sonora = pd.json_normalize(r.json()).T
resumen_sonora.columns = ['Valor']

resumen_sonora


,Valor
TotalGlobal,"7,960"
TotalDesaparecidos,"4,770"
TotalLocalizados,"3,190"
PorcentajeDesaparecidos,59.92 %
PorcentajeLocalizados,40.08 %
TotalSoloDesaparecidos,"4,730"
TotalSoloNoLocalizados,40
PorcentajeSoloDesaparecidos,99.16 %
PorcentajeSoloNoLocalizados,0.84 %
TotalLocalizadosCV,"2,929"


### Ejercicio

Probar con diferentes consultas y tratar de inferir los valores que pueden tomar (o buscarlas en la documentación de la API) las diferentes variables que pueden servir para encontrar búsquedas más específicas.

Por ejemplo, ¿Como podríamos consultar las estadísticas sobre mujeres desaparecidas en el municipio de Cajeme?

In [ ]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoMunicipio" # Seleccionamos Municipio

DATA = {
  "titulo":"",
  "subtitulo": "",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idEstado":"26", # Sonora = 26
  "idMunicipio":"18", # Cajeme = 18
  "idColonia":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
  "idDelito":"0"
}
r = main_session.post(TARGET_URL, json = DATA)

res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Municipio'] = res['XAxisCategories']
por_muni = pd.DataFrame(datos)

por_muni


,Hombre,Mujer,Indeterminado,Municipio
0,649,317,0,CAJEME


## Personas desaparecidas por sexo y colonia

El `endpoint` **BarChartSexoColonia** está diseñado para generar gráficas, pero nos permite extraer información, si la sabemos formatear.

Vamos viendo un ejemplo:


In [ ]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR COLONIAS - HERMOSILLO",
  "idEstado":"26",
  "idMunicipio":"30",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}
r = main_session.post(TARGET_URL, json = DATA)

res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Colonia'] = res['XAxisCategories']
por_colonia = pd.DataFrame(datos)
por_colonia.index = por_colonia.Colonia

por_colonia


,Hombre,Mujer,Indeterminado,Colonia
Colonia,,,,
VILLA HERMOSA,0,4,0,VILLA HERMOSA
MIGUEL ALEMÁN CENTRO,11,6,0,MIGUEL ALEMÁN CENTRO
QUINTA EMILIA,2,0,0,QUINTA EMILIA
ARBOLEDAS,1,1,0,ARBOLEDAS
SAHUARO INDECO,0,2,0,SAHUARO INDECO
UNIVERSIDAD,1,0,0,UNIVERSIDAD
JESÚS GARCIA,7,3,0,JESÚS GARCIA
MESA DEL SERI,1,1,0,MESA DEL SERI
NUEVA PALMIRA,1,0,0,NUEVA PALMIRA


### Ejercicio

¿Como podemos sacar lo que pasa en todo el estado, por municipios y por colonias? Intentalo.

Yo aquí creé un DataFrame con los municipios de Sonora y las mujeres y hombres desaparecidos, no supe como añadir las colonias. La idea suena fácil, tener un dataframe con los desaparecidos de todas las colonias de todos los municipios de Sonora, pero al no saber usar bien la api o no tener mucha información no es tan sencillo.

In [ ]:
API_HOST = "https://versionpublicarnpdno.segob.gob.mx/"
API_SOCIODEOGRAFICOS_ROOT = "Sociodemografico/"
API_CATALAGO_ROOT = "Catalogo/"

ENDPOINT_CATALOGO_EDO = "Estados/"
ENDPOINT_CATALOGO_MUN = "Municipios/"
ENDPOINT_CATALOGO_COL = "Colonias/"

# Before doing anything, we must make a dummy request to the index in order to get the propper cookies
main_session = requests.Session()
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index")
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/ContextoGeneral")

TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoMunicipio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR MUNICIPIOS - SONORA",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}

r = main_session.post(TARGET_URL, json=DATA)

res_estatal = r.json()
# Dic con Hombre:[], Mujer=[], Indeterminado=[]
datos_estatal = {serie['name']: serie['data'] for serie in res_estatal['Series']}

datos_estatal['Municipio'] = res_estatal['XAxisCategories']

por_municipio = pd.DataFrame(datos_estatal)
por_municipio


,Hombre,Mujer,Indeterminado,Municipio
0,1231,722,0,HERMOSILLO
1,669,403,0,NOGALES
2,649,317,0,CAJEME
3,331,93,0,GUAYMAS
4,320,182,0,AGUA PRIETA
5,286,83,0,CABORCA
6,248,109,0,PUERTO PEÑASCO
7,225,168,1,SAN LUIS RÍO COLORADO
8,177,64,0,NAVOJOA
9,171,24,0,GENERAL PLUTARCO ELÍAS CALLES


## Información por sexo y por año

Tambien se puede encontrar información por sexo y por año utilizando otro `endpoint`: **AreaChartSexoAnio**

Sin mas choro, vamos a ver como se usa, otra vez con el estado de Sonora:

In [ ]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR AÑO EN EL ESTADO DE SONORA",
  "idEstado":"26",
  "idMunicipio":"0",
  "idColonia":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "idEstatusVictima":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}

r = main_session.post(TARGET_URL, json = DATA)

res = r.json()

datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Fecha'] = res['XAxisCategories']

por_fecha = pd.DataFrame(datos)
por_fecha['Fecha'] = pd.to_numeric(por_fecha.Fecha, errors='coerce')
por_fecha.index = por_fecha.Fecha

por_fecha


,Hombre,Mujer,Indeterminado,Fecha
Fecha,,,,
NaN,127,45,5,NaN
1974.0,3,0,0,1974.0
1977.0,1,0,0,1977.0
1978.0,2,0,0,1978.0
1980.0,1,0,0,1980.0
1981.0,7,1,0,1981.0
1982.0,1,0,0,1982.0
1989.0,1,0,0,1989.0
1994.0,2,0,0,1994.0


### Ejercicio

¿Se puede hacer por municipio? ¿En forma programática? ¿Para algun caso especial? Intentalo

Sí lo podemos hacer por municipio, en este caso yo escogí Hermosillo. También podemos escoger algun caso especial como la circunstancia, yo puse la de gente que salió de casa a realizar alguna actividad y ya no regresó.

In [ ]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

DATA = {
  "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
  "subtitulo":"POR AÑO EN EL ESTADO DE SONORA",
  "idEstado":"26",
  "idMunicipio":"30", # Hermosillo = 30
  "idColonia":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "idEstatusVictima":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"11", # SALIÓ DE CASA A REALIZAR ALGUNA ACTIVIDAD Y YA NO REGRESO = 11
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}

r = main_session.post(TARGET_URL, json = DATA)

res = r.json()

datos = {serie['name']: serie['data'] for serie in res['Series']}
datos['Fecha'] = res['XAxisCategories']

por_fecha = pd.DataFrame(datos)
por_fecha['Fecha'] = pd.to_numeric(por_fecha.Fecha, errors='coerce')
por_fecha.index = por_fecha.Fecha

por_fecha

,Hombre,Mujer,Indeterminado,Fecha
Fecha,,,,
NaN,2,0,0,NaN
2017.0,1,0,0,2017.0
2018.0,3,0,0,2018.0
2019.0,4,0,0,2019.0
2020.0,9,1,0,2020.0
2021.0,9,3,0,2021.0
2022.0,6,3,0,2022.0
2023.0,12,4,0,2023.0
2024.0,49,11,0,2024.0


### Ejercicio

Extrae alguna información del conjunto de tados que pienses que es relevante, y explica porqué.

Aquí quise hacer un dataframe con las colonias de Hermosillo y ver el número de desaparecidos por cada circunstancia. Tuve algunas ideas pero no pude llegar 100% a lo que quería, se me complicó a la hora de organizar los datos en el dataframe, pero sí pude obtenerlos, nomas que no acomodarlos como se debe.
Me pareció interesante ver cuales son las circunstancias de desaparición en las distintas colonias de Hermosillo para analizar cuales son las más comunes y si hay algún patrón de desapariciones en ciertas colonias cercanas entre ellas. Creo que para las personas que trabajan buscando a esta gente, puede llegar a ser de gran utilidad tener información de la cual pueden sacar conclusiones para preever estos tipos de desapariciones.

In [164]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"

DATA = {
  "titulo":"",
  "subtitulo":"POR COLONIAS - HERMOSILLO",
  "idEstado":"26",
  "idMunicipio":"30",
  "idColonia":"0",
  "idEstatusVictima":"0",
  "idHipotesisNoLocalizacion":"0",
  "idDelito":"0",
  "fechaInicio":"",
  "fechaFin":"",
  "mostrarFechaNula":"0",
  "idNacionalidad":"0",
  "edadInicio":"",
  "edadFin":"",
  "mostrarEdadNula":"0",
  "idHipotesis":"",
  "idMedioConocimiento":"",
  "idCircunstancia":"0",
  "tieneDiscapacidad":"",
  "idTipoDiscapacidad":"0",
  "idEtnia":"0",
  "idLengua":"0",
  "idReligion":"",
  "esMigrante":"",
  "idEstatusMigratorio":"0",
  "esLgbttti":"",
  "esServidorPublico":"",
  "esDefensorDH":"",
  "esPeriodista":"",
  "esSindicalista":"",
  "esONG":"",
}

copy_data = DATA.copy()
lista = []
datos={}

for i in range(0,18):
  if i <= 9:

    copy_data["idCircunstancia"] = f'{i}'
    r = main_session.post(TARGET_URL, json = copy_data)
    res = r.json()

    datos['Circunstancia'] = i
    for serie in res['XAxisCategories']:
      datos['Colonia'] = res['XAxisCategories']
      for i in range(3):
        for num in res['Series'][i]['data']:
          datos[res['Series'][i]['name']] = res['Series'][i]['data']


    lista.append(datos)

  elif i >= 10:
    i += 1

    datos['Circunstancia'] = i
    copy_data["idCircunstancia"] = f'{i}'
    r = main_session.post(TARGET_URL, json = copy_data)
    res = r.json()

    for serie in res['XAxisCategories']:
      datos['Colonia'] = res['XAxisCategories']
      for i in range(3):
        for num in res['Series'][i]['data']:
          datos[res['Series'][i]['name']] = res['Series'][i]['data']

      #print(datos)

lista.append(datos)

print(lista)
df = pd.DataFrame(lista)

#por_colonia.index = por_colonia.Colonia

df


[{'Circunstancia': 18, 'Colonia': ['QUINTA EMILIA', 'SAHUARO INDECO', 'JESÚS GARCIA', 'MIGUEL ALEMÁN CENTRO', 'SAN JOSE DE LAS MINITAS', 'ARBOLEDAS', 'MIGUEL HIDALGO', 'SAN BENITO', 'LA MANGA', 'HERMOSILLO CENTRO', 'LOMAS DE MADRID', 'LAS VILLAS', 'CAFÉ COMBATE', 'LAS MINITAS', 'URBI VILLA DEL PRADO', 'INVASIÓN ALTARES', 'ARCO IRIS', 'BUGAMBILIA', 'CARMEN SERDÁN', 'BAHÍA DE KINO CENTRO', 'ADOLFO DE LA HUERTA', 'PUEBLITOS', 'VILLA SONORA', 'LAS GRANJAS', 'LOS OLIVOS INDEUR', 'NORBERTO ORTEGA', 'VILLA MERCEDES', 'JORGE VALDEZ MUÑOZ', 'RENACIMIENTO', 'LEY 57'], 'Hombre': [1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 3, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0], 'Mujer': [0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1], 'Indeterminado': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'Circunstancia': 18, 'Colonia': ['QUINTA EMILIA', 'SAHUARO INDECO', 'JESÚS GARCIA', 'MIGUEL ALEMÁN CENTRO', 'SAN JOSE DE 

,Circunstancia,Colonia,Hombre,Mujer,Indeterminado
0,18,"[QUINTA EMILIA, SAHUARO INDECO, JESÚS GARCIA, ...","[1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, ...","[0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,18,"[QUINTA EMILIA, SAHUARO INDECO, JESÚS GARCIA, ...","[1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, ...","[0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,18,"[QUINTA EMILIA, SAHUARO INDECO, JESÚS GARCIA, ...","[1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, ...","[0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,18,"[QUINTA EMILIA, SAHUARO INDECO, JESÚS GARCIA, ...","[1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, ...","[0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,18,"[QUINTA EMILIA, SAHUARO INDECO, JESÚS GARCIA, ...","[1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, ...","[0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,18,"[QUINTA EMILIA, SAHUARO INDECO, JESÚS GARCIA, ...","[1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, ...","[0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,18,"[QUINTA EMILIA, SAHUARO INDECO, JESÚS GARCIA, ...","[1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, ...","[0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,18,"[QUINTA EMILIA, SAHUARO INDECO, JESÚS GARCIA, ...","[1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, ...","[0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,18,"[QUINTA EMILIA, SAHUARO INDECO, JESÚS GARCIA, ...","[1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, ...","[0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,18,"[QUINTA EMILIA, SAHUARO INDECO, JESÚS GARCIA, ...","[1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, ...","[0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Practicando a ser investigador de APIs

Ahora te pido que revises si puedes encontrar otros endpoints para recuperar mas información de las bases que no se encuentran liberadas. Puede ser en la misma página, o en blogs o revisando código. Agrega en esta libreta la documentación (o enlaces a dicha documentación) y un ejemplo de uso de una API pobremente documentada.

No me dio tiempo a adentrarme bien a este ejercicio pero pienso continuarlo en cuanto pueda.